In [ ]:
import mne
import numpy as np
import pandas as pd
from mne.preprocessing.nirs import beer_lambert_law
from mne.preprocessing.nirs import optical_density
from sklearn.preprocessing import MinMaxScaler
import sched, time

In [ ]:
snirf_path = './task/2023-06-08_004.snirf'
raw_intensity = mne.io.read_raw_snirf(snirf_path, verbose=True, preload=False)

# resample to one Hz
raw_intensity.load_data().resample(1.0, npad="auto")

In [ ]:
# Define the motor unit to channel mapping
mu_0 = ['S5_D3 hbo', 'S5_D6 hbo']
mu_1 = ['S8_D6 hbo']
mu_2 = ['S8_D6 hbo']
mu_3 = ['S6_D5 hbo', 'S7_D5 hbo', 'S6_D7 hbo', 'S7_D4 hbo']
mu_4 = ['S6_D5 hbo', 'S7_D5 hbo', 'S6_D7 hbo', 'S7_D4 hbo']
mu_12 = ['S3_D1 hbo', 'S3_D2 hbo', 'S4_D2 hbo', 'S1_D1 hbo', 'S1_D2 hbo']
mu_13 = ['S3_D1 hbo', 'S3_D2 hbo', 'S4_D2 hbo', 'S1_D1 hbo', 'S1_D2 hbo']
mu_14 = ['S2_D3 hbo']
mu_15 = ['S5_D3 hbo', 'S2_D3 hbo']


In [ ]:
raw_od = optical_density(raw_intensity)
raw_haemo = beer_lambert_law(raw_od, ppf=0.1)
ch_names = raw_haemo.info['ch_names']
# ch_names = ['0'] + ch_names

# Converts the snirf data into a CSV
data = pd.DataFrame(raw_haemo.get_data())

#Normalize the data
np_data = data.to_numpy()
scaler = MinMaxScaler(feature_range=(0, 100))
norm_data = scaler.fit_transform(np_data)
df = pd.DataFrame(norm_data)

# Add the channel names
df.insert(0, 'ch_names', ch_names)
df.to_csv('hemo_data.csv')


In [56]:

# Run through the dataframe and ouput to motor units
starttime = time.time()
print(starttime)
while True:
    # print("tick")
    time.sleep(60.0 - ((time.time() - starttime) % 60.0))
    print('tick')
